In [52]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats
plt.style.use("ggplot")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
#plt.rcParams['font.family'] = 'IPAexGothic'

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす
import pickle
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import collections
import gensim
import random

model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [53]:
f = open("../fact_relation_vec_list.binaryfile",'rb')
#f = open("../kansei_relation_vec_list.binaryfile",'rb')
mean_vec_list = pickle.load(f)

mean_vec_list = random.sample(mean_vec_list, int(len(mean_vec_list)/20))
len(mean_vec_list)

3696

In [54]:
# (1) Google Spread Sheetsにアクセス
def connect_gspread(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1
    return worksheet

In [55]:
def result_vec_list_generator(imput_word_list):
    result_vec_list  = []

    for w in imput_word_list:

        try:
            w_to_v = model.wv[w]

            for mean_vec in mean_vec_list:
                result_vec_plus = w_to_v - mean_vec 
                result_vec_minus =   w_to_v + mean_vec 
    
                result_vec_list.append(result_vec_plus)
                result_vec_list.append(result_vec_minus)
        except:
            print("Error", w)

    return result_vec_list

In [56]:
# 関係ベクトルと類似度が高い単語をn個上位表示
def output_word_list_generator(result_vec_list):
    output_word_list = []
    max_n = 2

    for vec in result_vec_list:
        most_similar = np.array(list(model.wv.most_similar([vec], [], max_n)))
        #output_word_list.extend(most_similar[:,0].tolist())
        
        a = most_similar[:,1]
        for cos in a:
            if float(cos) >= 0.85: #コサイン類似度が0.8以下を除去
                output_word_list.extend(most_similar[:,0].tolist())
                
    return output_word_list

In [57]:
def ggg(list_taple):
    l = []
    for taple in list_taple:
        a = taple[0]
        l.append(a)
    return l

In [58]:
# ここでjsonfile名と2-2で用意したkeyを入力
jsonf = "quickstart-1595337771922-22c2ceb47d3c.json"
spread_sheet_key = "1WKpQ4sxL_IP9Fm6551xDRfhxhDLzmgGAG3FCzDFmamI"
worksheet = connect_gspread(jsonf,spread_sheet_key)

In [59]:
# スプレッドシートのデータを取得するコード
# {"answer":[imput_word_list], ...}

data_dic = {}

for i in range(2, 33): #すプレッソシートの行数
    answers_list = worksheet.row_values(i)
    imput_word_list = answers_list[2:]
    answer_word = answers_list[1]
    
    data_dic[answer_word] = imput_word_list

In [61]:
count = 0

for answer_word, imput_word_list in data_dic.items():
    print(answer_word)
    print(imput_word_list)
    
    result_vec_list = result_vec_list_generator(imput_word_list)
    output_word_list = output_word_list_generator(result_vec_list)
    
    for imput_word in imput_word_list:
        output_word_list = [output_word for output_word in output_word_list if output_word != imput_word]
    
    c = collections.Counter(output_word_list)
    dic = dict(c)
    dic2 = sorted(dic.items(), key=lambda x:x[1], reverse=True)
    l = ggg(dic2[:30])
    print(l)
    
    if answer_word in l: 
        print("成功**************************")
        
        count += 1
    
    print("　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　")
    
print("正答率", count/len(data_dic))

LIXIL
['トステム', 'INAX', '新日軽', 'サンウエーブ工業', '東洋エクステリア', 'YKK', '大建工業']
Error サンウエーブ工業
Error 東洋エクステリア
Error 大建工業


KeyboardInterrupt: 

In [ ]:
"""
for i in range(2, 33): #すプレッソシートの行数
    answers_list = worksheet.row_values(i)
    imput_word_list = answers_list[2:]
    answer_word = answers_list[1]
    print("answer", answer_word)
    print("imput", imput_word_list)
    
    result_vec_list = result_vec_list_generator(imput_word_list)
    output_word_list = output_word_list_generator(result_vec_list)
    
    for imput_word in imput_word_list:
        output_word_list = [output_word for output_word in output_word_list if output_word != imput_word]
    
    c = collections.Counter(output_word_list)
    dic = dict(c)
    dic2 = sorted(dic.items(), key=lambda x:x[1], reverse=True)
    l = ggg(dic2[:30])
    print(l)
    
    if answer_word in l: 
        print("成功**************************")
    
    print("　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　")
"""

In [ ]:
# 複数の単語で類似度の高いものを上から１０個表示する
most_similar = np.array(list(model.wv.most_similar(imput_word_list, [], 10)))[:,0]
most_similar.tolist()